In [0]:
%sql
-- BRONZE
ALTER TABLE thebetty.bronze.finances_invoices SET TAGS (
  'layer'            = 'bronze',
  'source_system'    = 'Navy ERP',
  'ingestion_method' = 'ADF via JDBC',
  'original_format'  = 'CSV',
  'owner'            = 'Data Engineering',
  'pii'              = 'no'
);

-- SILVER (joined/cleansed)
ALTER TABLE thebetty.silver.vendor_registry_clean SET TAGS (
  'layer'             = 'silver',
  'domain'            = 'finance',
  'dq_last_validated' = '2025-08-10',
  'dq_validation_score' = '98',
  'pii'               = 'no'
);

ALTER TABLE thebetty.silver.finance_with_vendor_info SET TAGS (
  'layer'      = 'silver',
  'domain'     = 'finance',
  'use_case'   = 'vendor spend + compliance analytics',
  'pii'        = 'no',
  'security_level' = 'internal',
  'compliance' = 'SOX'
);

-- GOLD (presentation)
ALTER TABLE thebetty.gold.final_vendor_summary SET TAGS (
  'layer'       = 'gold',
  'domain'      = 'finance',
  'use_case'    = 'vendor compliance reporting',
  'owner'       = 'Finance Analytics Team',
  'pii'         = 'no',
  'pipeline_name' = 'pipeline1_batch_delta'
);

-- GOLD run log table (if you want it to appear in lineage w/ context)
ALTER TABLE thebetty.gold.final_vendor_summary_runs SET TAGS (
  'layer'       = 'gold',
  'domain'      = 'governance',
  'use_case'    = 'pipeline run audit',
  'owner'       = 'Data Engineering'
);

-- OPTIONAL: a few column-level tags to make column lineage richer
ALTER TABLE thebetty.gold.final_vendor_summary
  ALTER COLUMN vendor_id        SET TAGS ('semantic_role'='business_key');
ALTER TABLE thebetty.gold.final_vendor_summary
  ALTER COLUMN compliance_score SET TAGS ('semantic_role'='quality_metric');
ALTER TABLE thebetty.gold.final_vendor_summary
  ALTER COLUMN onwatchlist      SET TAGS ('semantic_role'='risk_flag');
